In [5]:
import nuclio 
import mlrun
import os

In [88]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


In [2]:
# nuclio: start-code

In [6]:
import mlrun.feature_store as fs
import mlrun
import datetime
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import requests
import json
import numpy as np

In [61]:
def modify_data(context,ticker_data):
    ticker_to_int = {'GOOGL' : 0,'MSFT' : 1,'AMZN' : 2,'AAPL' : 3,'INTC' : 4}
    ticker_data["ticker"] = ticker_data["ticker"].apply(lambda x: ticker_to_int.get(x))
    for col in [x for x in ticker_data.columns if "Open" in x or "Close" in x or "High" in x or "Low" in x]:
        ticker_data[[col]] = context.priceMMS.fit_transform(ticker_data[[col]])
    for col in [x for x in ticker_data.columns if "Volume" in x]:
        ticker_data[[col]] = context.volumeMMS.fit_transform(ticker_data[[col]])
    for col in [x for x in ticker_data.columns if "Sentiment" in x]:
        ticker_data[[col]] = context.sentimentMMS.fit_transform(ticker_data[[col]])
        
    return ticker_data

In [62]:
def predict(context,data):
    event_data = {'inputs': data}
    endpoint = context.endpoint #+ "/v2/models/model2/predict"
    resp = requests.put(endpoint, json=json.dumps(event_data))
    return json.loads(resp.text)

In [75]:
def handler(context,event):
    start_time = datetime.datetime.now()-datetime.timedelta(5)
    data = fs.get_offline_features(context.vector,entity_timestamp_column="Datetime",
                           start_time = start_time,
                           end_time = datetime.datetime.now()).to_dataframe()
    all_ticker_predictions = []
    for ticker in context.sym_to_url.keys():
        ticker_df = data[data["ticker"] == ticker].fillna(0)
        ticker_df = ticker_df[-10:]
        modified_ticker = modify_data(context,ticker_df)
        context.logger.info(f"predicting stock price for {ticker}")
        stock_prediction = predict(context,modified_ticker.values.reshape(1,ticker_df.shape[0],ticker_df.shape[1]).tolist())
        stock_prediction = json.loads(stock_prediction["outputs"])
        all_ticker_predictions.append(context.priceMMS.inverse_transform(stock_prediction)[0][0])
                
    df = pd.DataFrame()
    df["ticker"] = context.sym_to_url.keys()
    df["predicted"] = all_ticker_predictions
    
    data = df.values.tolist()
    
    columns = [{'text': key, 'type': 'object'} for key in df.columns]
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [76]:
def init_context(context):
    context.logger.info("Initalizing context")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    features = ["stocks.Opens_min_1h",
            "stocks.Opens_max_1h",
            "stocks.Volumes_min_1h",
            "stocks.Volumes_max_1h",
            "stocks.Open",
            "stocks.High",
            "stocks.Low",
            "stocks.Close",
            "stocks.Volume",
            "stocks.ticker",
            "news.Sentiment"]
    
    vector = fs.FeatureVector("stocks-vec", features, description="stocks demo feature vector")
    vector.save()
    setattr(context,'vector',vector)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                'AAPL': 'apple-computer-inc', 'INTC' : 'intel-corp'}
    setattr(context, 'sym_to_url', sym_to_url)
    
    setattr(context,"priceMMS",MinMaxScaler(feature_range = (0, 1)))
    setattr(context,"volumeMMS",MinMaxScaler(feature_range = (0, 1)))
    setattr(context,"sentimentMMS",MinMaxScaler(feature_range = (0, 1)))
    
    setattr(context,"endpoint",os.getenv('endpoint', "http://default-tenant.app.app-lab-b117.iguazio-cd1.com:30585"))

In [77]:
# nuclio: end-code

In [78]:
#test locally
mlrun.set_environment(project="stocks-" + os.getenv('V3IO_USERNAME'))
init_context(context)
data = handler(context,"")

Python> 2021-07-01 11:33:39,119 [info] Initalizing context
Python> 2021-07-01 11:33:39,817 [info] predicting stock price for GOOGL
Python> 2021-07-01 11:33:39,897 [info] predicting stock price for MSFT
Python> 2021-07-01 11:33:39,966 [info] predicting stock price for AMZN
Python> 2021-07-01 11:33:40,034 [info] predicting stock price for AAPL
Python> 2021-07-01 11:33:40,104 [info] predicting stock price for INTC


In [79]:
data

[{'columns': [{'text': 'ticker', 'type': 'object'},
   {'text': 'predicted', 'type': 'object'}],
  'rows': [['GOOGL', 2440.325043225952],
   ['MSFT', 270.94796436812067],
   ['AMZN', 3442.4882586227754],
   ['AAPL', 137.11272962329258],
   ['INTC', 56.09725701435514]],
  'type': 'table'}]

In [58]:
#test remotly
# test remote deployment
from mlrun import code_to_function
import os
fn = code_to_function('rnn_model_prediction',
                      handler='handler')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : "stocks-" + os.getenv('V3IO_USERNAME'),
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [ ]:
addr = fn.deploy(project="stocks-" + os.getenv('V3IO_USERNAME'))

In [ ]:
!curl {addr}